**https://towardsdatascience.com/instance-hardness-threshold-an-undersampling-method-to-tackle-imbalanced-classification-problems-6d80f91f0581**

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('creditcardfraud.zip', compression='zip', header=0, sep=',', quotechar='"')

In [3]:
df.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


In [8]:
y = df['Class']

In [13]:
X = df.drop('Class', axis=1)

In [15]:
y.shape, X.shape

((284807,), (284807, 30))

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=5, random_state=42)
dt.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [18]:
from sklearn.metrics import confusion_matrix

In [20]:
preds = dt.predict(X_test)

In [21]:
confusion_matrix(preds, y_test)

array([[56856,    21],
       [    8,    77]])

In [23]:
!pip install imblearn
from imblearn.under_sampling import InstanceHardnessThreshold
from collections import Counter

     |████████████████████████████████| 163 kB 1.2 MB/s eta 0:00:01


In [24]:
print('Original test dataset shape %s' % Counter(y_train))

Original test dataset shape Counter({0: 227451, 1: 394})


In [28]:
iht = InstanceHardnessThreshold(estimator=dt, sampling_strategy='majority', random_state=42)
X_train_res, y_train_res = iht.fit_resample(X_train, y_train)

In [31]:
print(f'Resized dataset shape %s.'% Counter(y_train_res))

Resized dataset shape Counter({0: 45493, 1: 394}).


In [32]:
dt_new = DecisionTreeClassifier(max_depth=5, random_state=42)
dt_new.fit(X_train_res, y_train_res)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [35]:
preds_new = dt_new.predict(X_test)

In [36]:
confusion_matrix(preds_new, y_test)

array([[56843,    19],
       [   21,    79]])

In [40]:
from sklearn.metrics import matthews_corrcoef, f1_score

In [38]:
matthews_corrcoef(preds, y_test)

0.843414022099618

In [39]:
matthews_corrcoef(preds_new, y_test)

0.7976689474605003

In [41]:
f1_score(preds, y_test)

0.8415300546448088

In [42]:
f1_score(preds_new, y_test)

0.7979797979797979